# BTAA Geoportal County Map Data Processing

In order to show what counties BTAA geoportal is pulling data from in the map, we need to keep track of the counties with open geospatial data records and update them plus their search links in the map. This Jupyter Notebook is meant to automate the process using arcpy and ArcGIS API for Python. 

The process contains two main parts:
- Check if there has newly added counties and add them to Active feature class
- Upload the geodatabase containing BTAA active counties to AGOL 

Here is the data structure:
- One geodatabase **BTAA.gdb** with two feature classes: **BTAA_State** and **BTAA_County** - *Remain untouched*
- One csv file **Geoportal.csv** storing the FIPs and search links of all active counties - *Update this!*
- One empty folder **BTAA_Active**

In [1]:
################### Manual items to change! ###################
## set the date updating the counties
ActionDate = '20201130'

## names of the main directory containing Geoportal.csv and active folder
## Windows:
root = r"C:\Users\Zhouy\Desktop\BTAA_County Map\ArcGIS JavaScript API\data"
## MAC or Linux:
## root = "C:\Users\Zhouy\Desktop\BTAA_County Map\ArcGIS JavaScript API\data"

## Step 1: Check if there has newly added counties and add them to Active feature class

### 1.1 Initialization

In [2]:
# Import modules
import arcpy
import os
import shutil

In [3]:
# Set current workspace
# Remeber to change it to your own
rawdata = root + "\\BTAA.gdb"
activeGDB = root + "\\Activegdb"

### 1.2 Build up the working geodatabase: BTAA_Active.gdb
This step is to import feature classes and csv file to the new geodatabse

In [4]:
# Create the output workspace - BTAA_Active_ActionDate.gdb
# Execute CreateFileGDB
arcpy.CreateFileGDB_management(activeGDB, "BTAA_Active_%s.gdb" % (ActionDate))
outputspace = activeGDB + "\\BTAA_Active_%s.gdb" % (ActionDate)

In [5]:
# Convert Geoportal.csv as table to geodatabase
inputTable = os.path.join(root, "Geoportal.csv")
arcpy.TableToGeodatabase_conversion(inputTable, outputspace)

<Result 'C:\\Users\\Zhouy\\Desktop\\BTAA_County Map\\ArcGIS JavaScript API\\data\\Activegdb\\BTAA_Active_20201130.gdb'>

In [6]:
# Convert feature classes to geodatabase
# in_features = [os.path.join(rawdata, "BTAA_County"), os.path.join(rawdata, "BTAA_State")]
in_features = os.path.join(rawdata, "BTAA_County")
# Execute FeatureClassToGeodatabase
arcpy.FeatureClassToGeodatabase_conversion(in_features, outputspace)

<Result 'C:\\Users\\Zhouy\\Desktop\\BTAA_County Map\\ArcGIS JavaScript API\\data\\Activegdb\\BTAA_Active_20201130.gdb'>

### 1.3 Join table to county feature class
This step is to filter counties with open data geoportals and copy them to a new feature class *BTAA_Active*

In [7]:
# Set working environment as BTAA_Active.gdb
arcpy.env.workspace = activeGDB + "\\BTAA_Active_%s.gdb" % (ActionDate)

In [8]:
Actionshp = root + "\\BTAA_Active\\BTAA_Active_%s" % (ActionDate)
if not os.path.exists(Actionshp):
    os.makedirs(Actionshp)

In [9]:
# Add a new field to table (duplicate the FIPS field but change the data type as text)
inTable = "Geoportal"
fieldName = "FIPS_new"
fieldLength = 5
expression = "!FIPS!"
 
# Execute AddField 
arcpy.AddField_management(inTable, fieldName, "TEXT", field_length = fieldLength)

# Execute CalculateField 
arcpy.CalculateField_management(inTable, fieldName, expression, "PYTHON3")

<Result 'C:\\Users\\Zhouy\\Desktop\\BTAA_County Map\\ArcGIS JavaScript API\\data\\Activegdb\\BTAA_Active_20201130.gdb\\Geoportal'>

In [10]:
# Join table to BTAA counties based on FIP and only keep the common fields
inFeatures = "BTAA_County"
joinTable = "Geoportal"
inputJoinField = "FIPS"
outputJoinField = "FIPS_new"
joinType = "KEEP_COMMON"
outFeature = "BTAA_Active"

# Execute AddJoin
BTAA_joined_table = arcpy.AddJoin_management(inFeatures, inputJoinField, joinTable, 
                                            outputJoinField, joinType)

# Copy the layer to a new permanent feature class - BTAA_Active
arcpy.CopyFeatures_management(BTAA_joined_table, outFeature)

<Result 'C:\\Users\\Zhouy\\Desktop\\BTAA_County Map\\ArcGIS JavaScript API\\data\\Activegdb\\BTAA_Active_20201130.gdb\\BTAA_Active'>

In [11]:
# Convert feature class to shapefile for further upload
# arcpy.FeatureClassToShapefile_conversion("BTAA_State", os.path.join(root, "BTAA_State"))
arcpy.FeatureClassToShapefile_conversion("BTAA_Active", Actionshp)

<Result 'C:\\Users\\Zhouy\\Desktop\\BTAA_County Map\\ArcGIS JavaScript API\\data\\BTAA_Active\\BTAA_Active_20201130'>

## Step 2: Load Feature Classes to ArcGIS Online

### 2.1 Initialization

In [12]:
# Import GIS module hosted by AGOL
from arcgis.gis import GIS

In [13]:
# Create a GIS object
gis = GIS("Pro")

### 2.2 Publish the feature class as a web layer

In [14]:
# Specify the properties file in a dictionary
active_properties = {"type":"Shapefile", "title":"BTAA_Active_WI", "tags":"BTAA"}
# Note that the uploaded item needs to be zipped!
unzipped_file = root +"\\BTAA_Active\\BTAA_Active_%s" % (ActionDate)
shutil.make_archive(unzipped_file, 'zip', unzipped_file)
data_file_location = root + "\\BTAA_Active\\BTAA_Active_%s.zip" % (ActionDate)
# Add the content to AGOL
active_shp = gis.content.add(active_properties, data = data_file_location)
# Publish the file
active_feature_layer_item = active_shp.publish()

active_feature_layer_item

<Item title:"BTAA_Active_WI" type:Feature Layer Collection owner:zhou1439_UMN>

In [15]:
# state_properties = {"type":"Shapefile", "title":"BTAA_State", "tags":"BTAA"}
# data_file_location = root + "\\BTAA_State\\BTAA_State.zip"
# state_shp = gis.content.add(state_properties, data = data_file_location)
# state_feature_layer_item = state_shp.publish()

# state_feature_layer_item

### 2.3 Search AGOL to find the content published before

> Remeber to copy and paste the itemID of **BTAA_Active** to ***main.js***. That's how we display the latest county data. 

In [16]:
# Access the dataset published before 
search_active = gis.content.search(query='title: "BTAA_Active_WI" AND type: "Feature Service"')
# Retrieve the feature layer collection item from the list of results
active_feature_layer_item = search_active[0]
# Update the sharing permissions of the layer so that it's available to everyone
active_feature_layer_item.share(everyone=True)

{'results': [{'itemId': 'd570c37116e34791a6a169c12678c9d7',
   'success': True,
   'notSharedWith': []}]}

In [17]:
# search_state = gis.content.search(query='title: "BTAA_State" AND type: "Feature Service"')
# state_feature_layer_item = search_state[0]
# state_feature_layer_item.share(everyone=True)